In [81]:
import pandas as pd
import numpy as np

In [82]:
df = pd.read_pickle("data_sources/filtered/ntsb.pkl")
df.sample(10)

,Vehicles.VehicleNumber,Vehicles.DamageLevel,Vehicles.ExplosionType,Vehicles.FireType,Vehicles.SerialNumber,Vehicles.Make,Vehicles.Model,Vehicles.NumberOfEngines,Vehicles.RegistrationNumber,Vehicles.FlightOperationType,...,Country,EventDate,State,Agency,EventType,AirportId,AirportName,Latitude,Longitude,TotalInjuryCount
11914,1,substantial,none,none,17276639,cessna,172p,1,n9853l,inst,...,usa,2011-10-06 01:30:00,None,ntsb,acc,pgum,guam international airport,13.483611,144.791381,0
20610,1,substantial,on-ground,on-ground,18263383,cessna,182p,1,n9187g,pers,...,usa,2004-12-19 10:48:00,ca,ntsb,acc,ful,fullerton municipal airport,33.888053,-118.023612,2
8973,1,substantial,none,none,5109,christen industries inc,pitts s,1,n50xv,inst,...,usa,2014-08-12 11:52:00,fl,ntsb,acc,None,None,27.232221,-82.524169,2
18729,1,substantial,none,none,2235,aviat,a-1b,1,n166ma,pers,...,usa,2006-05-26 20:05:00,tx,ntsb,acc,None,None,28.941667,-96.536941,2
4934,1,substantial,none,none,18-8140,piper,pa18,1,n5373y,pers,...,usa,2018-08-31 19:00:00,ak,ntsb,acc,None,None,62.095554,-148.212219,0
7111,1,substantial,none,none,172s8937,cessna,172,1,n128rm,inst,...,usa,2016-07-18 16:00:00,mo,ntsb,acc,sus,spirit of st louis,38.660278,-90.645835,0
15775,1,substantial,none,none,12-2288,piper,pa-12,1,n3424m,bant,...,usa,2008-08-01 15:46:00,sc,ntsb,acc,cre,grand strand airport,33.811668,-78.723892,1
17558,1,substantial,NaN,none,28-7990479,piper,pa-28-181,1,n2834u,pers,...,usa,2007-05-04 20:30:00,nv,ntsb,acc,las,mc carran intl,36.080001,-115.152221,1
17612,1,none,none,none,23724,boeing,757-24apf,2,n402up,None,...,usa,2007-04-24 07:20:00,ca,ntsb,acc,bur,bob hope airport,34.200553,-118.358612,1
17014,1,destroyed,unknown,unknown,32-7200030,piper,pa-32-260,1,n5067t,pers,...,usa,2007-08-25 14:15:00,ny,ntsb,acc,None,None,40.700000,-72.583335,1


In [ ]:

ntsb_copied = df.copy()
ntsb_copied = ntsb_copied.rename(columns={"EventDate": "Date", "NtsbNumber": "ID", "State": "Location"}) # for schema matching 

seed = 10
np.random.seed(seed)

n = len(ntsb_copied)

# select 40% indices for later use
n_forty = int(np.floor(0.4 * n))
random_indices = np.random.choice(ntsb_copied.index, n_forty, replace=False)
print(f"random_indices count (40%): {len(random_indices)}")  # Should be ~0.4 * n

# select half of these indices to assign NaN in 'ID'
n_missing = int(np.floor(0.5 * n_forty))
missing_indices = np.random.choice(random_indices, n_missing, replace=False)

# assign NaN only to these missing_indices
ntsb_copied.loc[missing_indices, "ID"] = np.nan  # for slot filling

print(f"Number of NaNs assigned (should be 20% of n): {len(missing_indices)}")

# count total NaNs in 'ID' (including existing NaNs)
total_nans = ntsb_copied['ID'].isna().sum()
print(f"Total NaNs in 'ID' column: {total_nans}")

conflict_indices = np.setdiff1d(random_indices, missing_indices)

# transform strings for conflict resolution
for index, row in ntsb_copied.iterrows():
    airport = row["AirportName"]
    if pd.notna(airport):
        result = ' '.join([word[0] + '.' for word in airport.split()])
        ntsb_copied.loc[index, "AirportName"] = result

random_indices count (40%): 9361
Number of NaNs assigned (should be 20% of n): 4680
Total NaNs in 'ID' column: 4680


11914       g. i. a.
20610       f. m. a.
8973            None
18729           None
4934            None
7111     s. o. s. l.
15775       g. s. a.
17558       m. c. i.
17612       b. h. a.
17014           None
Name: AirportName, dtype: object